In [ ]:
# Multi-Agent AI Project 1 – Ethical Multi-Agent Simulation with Karma Logic & Consensus

import random
import json

# Step 1: Define Agent Class
class WaterAgent:
    def __init__(self, name, need_liters, priority):
        self.name = name
        self.need_liters = need_liters
        self.priority = priority
        self.allocated = 0
        self.sacrificed = False
        self.karma = 0

    def request_water(self):
        return self.need_liters

    def receive_water(self, liters):
        self.allocated = liters

    def update_karma(self):
        if self.allocated == 0:
            self.karma += 2
        elif self.allocated < self.need_liters:
            self.karma += 1
        elif self.allocated == self.need_liters:
            self.karma += 0
        elif self.allocated > self.need_liters:
            self.karma -= 1

    def to_dict(self):
        return {
            "name": self.name,
            "need_liters": self.need_liters,
            "priority": self.priority,
            "allocated": self.allocated,
            "sacrificed": self.sacrificed,
            "karma": self.karma
        }

# Step 2: Define Environment
class WaterTankEnvironment:
    def __init__(self, total_water_liters):
        self.total_water = total_water_liters
        self.agents = []
        self.allocation_log = []

    def add_agent(self, agent):
        self.agents.append(agent)

    def fair_allocate(self):
        self.agents.sort(key=lambda x: (x.priority, -x.need_liters, -x.karma))

        remaining_water = self.total_water
        for agent in self.agents:
            req = agent.request_water()
            alloc = min(req, remaining_water)

            if alloc < req:
                agent.sacrificed = True

            agent.receive_water(alloc)
            agent.update_karma()
            remaining_water -= alloc
            self.allocation_log.append(agent.to_dict())

    def reset_agents(self):
        for agent in self.agents:
            agent.allocated = 0
            agent.sacrificed = False

    def print_allocation_summary(self, round_num):
        print(f"\n🔁 Round {round_num} - Water Allocation Summary:")
        for log in self.allocation_log:
            print(f"Agent {log['name']}: Requested {log['need_liters']}L, Received {log['allocated']}L, Sacrificed: {log['sacrificed']}, Karma: {log['karma']}")

    def export_log(self, round_num):
        filename = f"multi_agent_allocation_log_round{round_num}.json"
        with open(filename, "w") as f:
            json.dump(self.allocation_log, f, indent=2)

# Step 3: Run Multi-Round Consensus Simulation
if __name__ == "__main__":
    print("🌱 Running Ethical Multi-Agent Simulation with Karma + Consensus...\n")

    env = WaterTankEnvironment(total_water_liters=100)
    env.add_agent(WaterAgent("Village A", 40, 1))
    env.add_agent(WaterAgent("Village B", 30, 2))
    env.add_agent(WaterAgent("Village C", 50, 3))
    env.add_agent(WaterAgent("Village D", 25, 1))

    NUM_ROUNDS = 3
    for round_num in range(1, NUM_ROUNDS + 1):
        print(f"\n=== Round {round_num} ===")
        env.allocation_log = []  # Reset log
        env.fair_allocate()
        env.print_allocation_summary(round_num)
        env.export_log(round_num)
        env.reset_agents()
        print("Karma carried into next round:\n")
        for agent in env.agents:
            print(f"  {agent.name} ➤ Karma: {agent.karma}")
